# NVIDIA Nemotron Nano 9B v2 - Interactive Demo

## Overview

This notebook provides an interactive demonstration of **NVIDIA Nemotron Nano 9B v2** capabilities using NVIDIA NIM. Experience the power of this compact 9B parameter model through hands-on examples.

## What You'll Explore

- 💬 **Conversational AI**: Natural language understanding and generation
- 🔧 **Function Calling**: Intelligent tool use and parameter extraction
- 🎭 **Roleplay**: Consistent persona-based interactions
- 🧩 **Reasoning**: Step-by-step problem solving

## Model Information

**Nemotron Nano 9B v2** specifications:
- **Parameters**: 9 Billion (optimized architecture)
- **Context Length**: 8K tokens
- **Inference Speed**: <100ms (optimized deployment)
- **License**: MIT (commercial use allowed)
- **Strengths**: Function calling, reasoning, code generation

## Prerequisites

- NVIDIA API key from [build.nvidia.com](https://build.nvidia.com/)
- Python 3.8+
- Internet connection

Let's get started! 🚀


## Setup

Install required packages and configure API access.


In [ ]:
# Install required packages
import sys
import subprocess

def install_package(package_name):
    """Install a package handling various environment scenarios"""
    try:
        # Try using pip module
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"✅ Successfully installed {package_name} using pip")
    except subprocess.CalledProcessError:
        try:
            # Try ensurepip to bootstrap pip first
            print("⚠️ pip not found, attempting to install pip...")
            subprocess.check_call([sys.executable, "-m", "ensurepip", "--default-pip"])
            subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip"])
            subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
            print(f"✅ Successfully installed pip and {package_name}")
        except:
            try:
                # Try conda as fallback
                subprocess.check_call(["conda", "install", "-y", package_name])
                print(f"✅ Successfully installed {package_name} using conda")
            except:
                print(f"❌ Failed to install {package_name}")
                print(f"Please run manually: {sys.executable} -m pip install {package_name}")
                print(f"Or: pip install {package_name}")
                raise

# Install openai
install_package("openai")


In [ ]:
# Import libraries
import os
from openai import OpenAI

print("✅ Setup complete!")


---

## API Configuration

Set your NVIDIA API key. Get one at [build.nvidia.com](https://build.nvidia.com/)


In [ ]:
# Configure API key
NVIDIA_API_KEY = "YOUR_API_KEY_HERE"  # Replace with your key
# Or use: NVIDIA_API_KEY = os.getenv("NVIDIA_API_KEY")

# Validate API key
if NVIDIA_API_KEY == "YOUR_API_KEY_HERE" or not NVIDIA_API_KEY:
    raise ValueError(
        "❌ ERROR: Please set your NVIDIA API key!\n\n"
        "Steps:\n"
        "1. Go to https://build.nvidia.com/\n"
        "2. Sign in and generate an API key\n"
        "3. Replace 'YOUR_API_KEY_HERE' above with your actual key\n"
        "4. Re-run this cell\n\n"
        "Example: NVIDIA_API_KEY = 'nvapi-xxxxxxxxxxxxx'"
    )

# Initialize client
client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=NVIDIA_API_KEY
)

print("✅ NVIDIA NIM client initialized!")
print(f"🔑 API key: {NVIDIA_API_KEY[:10]}...{NVIDIA_API_KEY[-4:]}")


## Test API Connection

Let's verify your API key works and check available models.


In [ ]:
# Test API connection and list available models
try:
    models = client.models.list()
    print("✅ API connection successful!\n")
    print("Available models:")
    for model in models.data:
        if "nemotron" in model.id.lower() or "llama" in model.id.lower():
            print(f"  - {model.id}")
except Exception as e:
    print(f"❌ API Error: {e}")
    print("\nTroubleshooting:")
    print("1. Go to https://build.nvidia.com/")
    print("2. Find 'Nemotron Nano' in the catalog")
    print("3. Click 'Get API Key' and accept terms")
    print("4. Make sure your API key has access to the model")


---

# Demo 1: Conversational AI 💬

Let's start with a simple conversation showcasing the model's natural language understanding.


In [ ]:
def chat(prompt, system_prompt="You are a helpful AI assistant.", stream=True):
    """Simple chat function with streaming support"""
    response = client.chat.completions.create(
        model="nvidia/nemotron-nano-9b-v2",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=1024,
        stream=stream
    )
    
    if stream:
        print("🤖 Nemotron: ", end="", flush=True)
        full_response = ""
        for chunk in response:
            if chunk.choices[0].delta.content:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        print("\n")
        return full_response
    else:
        return response.choices[0].message.content

# Test it out!
print("👤 You: Explain quantum computing in simple terms\n")
chat("Explain quantum computing in simple terms")


### Try Code Generation

The model excels at generating clean, functional code.


In [ ]:
print("👤 You: Write a Python function to calculate Fibonacci numbers\n")
chat("Write a Python function to calculate Fibonacci numbers with proper documentation")


---

# Demo 2: Function Calling 🔧

Nemotron Nano 9B v2 can intelligently decide which functions to call based on user requests.


In [ ]:
import json

# Define available functions
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "City name"},
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Perform mathematical calculations",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {"type": "string", "description": "Math expression to evaluate"}
                },
                "required": ["expression"]
            }
        }
    }
]

def demo_function_calling(user_query):
    """Demonstrate function calling"""
    print(f"👤 You: {user_query}\n")
    
    response = client.chat.completions.create(
        model="nvidia/nemotron-nano-9b-v2",
        messages=[{"role": "user", "content": user_query}],
        tools=tools,
        tool_choice="auto"
    )
    
    message = response.choices[0].message
    
    if message.tool_calls:
        print("🔧 Model Decision: Call a function\n")
        tool_call = message.tool_calls[0]
        print(f"📞 Function: {tool_call.function.name}")
        print(f"📝 Arguments: {tool_call.function.arguments}\n")
        
        # Pretty print the parsed arguments
        args = json.loads(tool_call.function.arguments)
        print("✨ Extracted Parameters:")
        for key, value in args.items():
            print(f"   • {key}: {value}")
    else:
        print(f"🤖 Response: {message.content}")
    print()

# Test different queries
demo_function_calling("What's the weather in Tokyo?")
demo_function_calling("Calculate 15% tip on $89.50")


---

# Demo 3: Roleplay Scenarios 🎭

The model can maintain consistent personas throughout conversations.


In [ ]:
def roleplay_demo(persona, user_message):
    """Demonstrate roleplay with different personas"""
    personas = {
        "tutor": "You are a patient, encouraging AI tutor who explains concepts clearly with examples.",
        "coder": "You are an expert programming assistant who writes clean, efficient code with best practices.",
        "writer": "You are a creative writer who crafts engaging narratives with vivid descriptions."
    }
    
    print(f"🎭 Persona: {persona.title()}")
    print(f"👤 You: {user_message}\n")
    
    chat(user_message, system_prompt=personas[persona])

# Try different personas
roleplay_demo("tutor", "I don't understand recursion. Can you help?")


In [ ]:
roleplay_demo("coder", "Write a function to reverse a linked list")


---

# Demo 4: Reasoning Challenges 🧩

Watch the model solve complex problems with step-by-step reasoning.


In [ ]:
reasoning_prompts = {
    "logic": "Three people are in a room. Alex is taller than Blake but shorter than Casey. Who is the tallest? Think step-by-step.",
    "math": "If you have a 3-gallon jug and a 5-gallon jug, how can you measure exactly 4 gallons? Explain your reasoning.",
    "lateral": "A man lives on the 15th floor. Every day he takes the elevator down. On the way back, he takes it to the 7th floor and walks the rest. Why? Provide logical reasoning."
}

def solve_puzzle(puzzle_type):
    """Demonstrate reasoning capabilities"""
    prompt = reasoning_prompts[puzzle_type]
    print(f"🧩 Puzzle Type: {puzzle_type.title()}")
    print(f"❓ Question: {prompt}\n")
    
    response = chat(prompt, system_prompt="You are a logical reasoning expert. Think step-by-step and show your work.", stream=False)
    print(f"🤖 Solution:\n{response}\n")
    print("="*80 + "\n")

# Solve different types of puzzles
solve_puzzle("logic")


In [ ]:
solve_puzzle("math")


---

# Multi-Turn Conversation

Build context over multiple exchanges.


In [ ]:
conversation = [
    {"role": "system", "content": "You are a helpful AI assistant."}
]

def continue_conversation(user_input):
    """Continue a multi-turn conversation"""
    conversation.append({"role": "user", "content": user_input})
    print(f"👤 You: {user_input}\n")
    
    response = client.chat.completions.create(
        model="nvidia/nemotron-nano-9b-v2",
        messages=conversation,
        temperature=0.7,
        max_tokens=512
    )
    
    assistant_msg = response.choices[0].message.content
    conversation.append({"role": "assistant", "content": assistant_msg})
    
    print(f"🤖 Nemotron: {assistant_msg}\n")
    print("="*80 + "\n")

# Have a conversation
continue_conversation("What are the benefits of edge AI deployment?")
continue_conversation("Can you give me specific examples in manufacturing?")
continue_conversation("What hardware would I need?")


---

# Summary

You've explored the key capabilities of **NVIDIA Nemotron Nano 9B v2**:

✅ **Conversational AI** - Natural, context-aware dialogue  
✅ **Function Calling** - Intelligent tool use and parameter extraction  
✅ **Roleplay** - Consistent persona-based interactions  
✅ **Reasoning** - Step-by-step problem solving  
✅ **Multi-turn Context** - Maintaining conversation history

## Next Steps

- 📘 Check out `Integrate_With_NIM.ipynb` for more advanced patterns
- 🚀 Deploy with NVIDIA NIM for production use
- 🔧 Customize functions and personas for your use case
- 📚 Read the [NVIDIA NIM Documentation](https://docs.nvidia.com/nim/)

## Resources

- [NVIDIA NIM](https://www.nvidia.com/en-us/ai/)
- [Nemotron Model Family](https://developer.nvidia.com/nemotron)
- [API Catalog](https://build.nvidia.com/)
- [Documentation](https://docs.nvidia.com/nim/)

---

**Built with ❤️ for the NVIDIA AI community**
